# Heart Tor Model

In [1]:
import numpy as np 


import matplotlib.pyplot as plt

import pickle



import os, sys
rootpath = os.path.join(os.getcwd(), '..')
sys.path.append(rootpath)


from scipy.sparse import diags, coo_matrix
from scipy import sparse
import jax.numpy as jnp

import jax
from jax import lax, vmap, jit
import jax.random as random
from jax.experimental import sparse
rootpath = os.path.join(os.getcwd(), '..')
codepath = os.path.join(os.getcwd(), '../codebase') #directory where you have the class file
sys.path.append(rootpath)
sys.path.append(codepath)
from codebase.FHN_model import * 


In [12]:
heart_toymodel = FHN_model( 
                 N=200,
                 organ='heart',

                 adjacency_seed=1000,
                 stimulus_time=2000)    

In [13]:
heart_toymodel.solve_with_EulerMaruyama_fori(T=10000,output_times=10000)

In [18]:
heart_arrhythmia = FHN_model(N=200,
                            organ='heart',
                            adjacency_seed=1000,
                            p=0.5,
                            stimulus_time=2000)
heart_asystole = FHN_model(N=200,
                            organ='heart',
                            adjacency_seed=1000,
                            p=0.7,
                            stimulus_time=2000)

In [20]:
heart_arrhythmia.solve_with_EulerMaruyama_fori(T=10000,output_times=10000)
heart_asystole.solve_with_EulerMaruyama_fori(T=10000,output_times=10000)

In [23]:
folder='data/simulations/V_p=0.pkl'
with open(folder, 'wb') as f:
    pickle.dump(np.array(heart_toymodel.vs), f)
folder='data/simulations/V_p=0.5.pkl'
with open(folder, 'wb') as f:
    pickle.dump(np.array(heart_arrhythmia.vs), f)
folder='data/simulations/V_p=0.7.pkl'
with open(folder, 'wb') as f:
    pickle.dump(np.array(heart_asystole.vs), f)

In [31]:
ECG_0=np.array(heart_toymodel.vs[ :, :].sum(axis=1)/1000)
ECG_05=np.array(heart_arrhythmia.vs[ :, :].sum(axis=1)/1000)
ECG_07=np.array(heart_asystole.vs[ :, :].sum(axis=1)/1000)
t=np.arange(0,10000,1)

In [39]:

np.savetxt('ECG_p=0.txt', np.column_stack((t, ECG_0)), header='t ECG', comments='')
np.savetxt('ECG_p=0.5.txt', np.column_stack((t, ECG_05)), header='t ECG', comments='')
np.savetxt('ECG_p=0.7.txt', np.column_stack((t, ECG_07)), header='t ECG', comments='')

In [56]:
def raster_plot(array):
    start_times = []
    durations = []
    indices = []
    for i in range(array.shape[0]):
        signal = array[i, :]
        start_time = np.where(np.diff(signal, prepend=0) == 1)[0]
        end_time = np.where(np.diff(signal, append=0) == -1)[0]
        for s, e in zip(start_time, end_time):
            start_times.append(s)
            durations.append(e - s)
            indices.append(i)

    start_times = np.array(start_times)
    durations = np.array(durations)
    indices = np.array(indices)
    return start_times, durations, indices


In [57]:

N=heart_toymodel.N
raster_0=np.where(heart_toymodel.vs > 0.8, 1, 0).reshape(10000, N,N).T
raster_0=raster_0[:, 20,:]

start_times_0, durations_0, indices_0 = raster_plot(raster_0)

raster_1=np.where(heart_arrhythmia.vs > 0.8, 1, 0).reshape(10000, N,N).T
raster_1=raster_1[:, 20,:]

start_times1, durations_1, indices_1 = raster_plot(raster_1)


raster_2=np.where(heart_asystole.vs > 0.8, 1, 0).reshape(10000, N,N).T
raster_2=raster_2[:, 20,:]

start_times_2, durations_2, indices_2 = raster_plot(raster_2)



In [ ]:
np.savetxt('Raster_plot_column_p=0.txt', np.column_stack((start_times_0,durations_0, indices_0)), header='times durations ids', comments='')
np.savetxt('Raster_plot_column_p=0.5.txt', np.column_stack((start_times1, durations_1,indices_1)), header='times durations ids', comments='')
np.savetxt('Raster_plot_column_p=0.7.txt', np.column_stack((start_times_2, durations_2,indices_2)), header='times durations ids', comments='')

In [59]:

N=heart_toymodel.N
raster_0=np.where(heart_toymodel.vs > 0.8, 1, 0).reshape(10000, N,N).T
raster_0=raster_0[20, :,:]

start_times_0, durations_0, indices_0 = raster_plot(raster_0)

raster_1=np.where(heart_arrhythmia.vs > 0.8, 1, 0).reshape(10000, N,N).T
raster_1=raster_1[20, :,:]

start_times1, durations_1, indices_1 = raster_plot(raster_1)


raster_2=np.where(heart_asystole.vs > 0.8, 1, 0).reshape(10000, N,N).T
raster_2=raster_2[20, :,:]

start_times_2, durations_2, indices_2 = raster_plot(raster_2)



In [61]:
np.savetxt('Raster_plot_row_p=0.txt', np.column_stack((start_times_0,durations_0, indices_0)), header='times durations ids', comments='')
np.savetxt('Raster_plot_row_p=0.5.txt', np.column_stack((start_times1, durations_1,indices_1)), header='times durations ids', comments='')
np.savetxt('Raster_plot_row_p=0.7.txt', np.column_stack((start_times_2, durations_2,indices_2)), header='times durations ids', comments='')

# Brain Toy model

In [63]:
Brain_healthy= FHN_model(N=40000,
                        organ='brain',
                        m=0.005,
                        sigma=0.04,
                        adjacency_seed=1000)
Brain_epilepsy= FHN_model(N=40000,
                        organ='brain',
                        m=0.145,
                        sigma=0.04,
                        adjacency_seed=1000)

c:\Users\pelli\Projects\perspective_heart_and_brain\notebooks_figures_paper\..\codebase\FHN_model.py:158: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(G, weight='weight')
c:\Users\pelli\Projects\perspective_heart_and_brain\notebooks_figures_paper\..\codebase\FHN_model.py:158: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(G, weight='weight')


In [64]:
Brain_healthy.solve_with_EulerMaruyama_fori(T=10000,output_times=10000)
Brain_epilepsy.solve_with_EulerMaruyama_fori(T=10000,output_times=10000)

In [65]:
folder='data/simulations/V_m=0.005.pkl'
with open(folder, 'wb') as f:
    pickle.dump(np.array(Brain_healthy.vs), f)
folder='data/simulations/V_m=0.145.pkl'
with open(folder, 'wb') as f:
    pickle.dump(np.array(Brain_healthy.vs), f)

In [66]:
EEG_005=np.array(Brain_healthy.vs[ :, :].sum(axis=1)/1000)
EEG_145=np.array(Brain_epilepsy.vs[ :, :].sum(axis=1)/1000)
t=np.arange(0,10000,1)

In [67]:

np.savetxt('EEG_m=0.005.txt', np.column_stack((t, EEG_005)), header='t EEG', comments='')
np.savetxt('EEG_m=0.145.txt', np.column_stack((t, EEG_145)), header='t EEG', comments='')


In [68]:

N=Brain_healthy.N
raster_0=np.where(Brain_healthy.vs > 0.5, 1, 0).T


start_times_0, durations_0, indices_0 = raster_plot(raster_0)

raster_1=np.where(Brain_epilepsy.vs > 0.5, 1, 0).T


start_times1, durations_1, indices_1 = raster_plot(raster_1)


In [69]:
np.savetxt('Raster_plot_m=0.005.txt', np.column_stack((start_times_0,durations_0, indices_0)), header='times durations ids', comments='')
np.savetxt('Raster_plot_m=0.145.txt', np.column_stack((start_times1, durations_1,indices_1)), header='times durations ids', comments='')
